In [6]:
import torchvision
import torch

from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Lambda

torch.cuda.is_available()

False

In [4]:
#modify the image from image to 784 dots
transform = Compose([
    ToTensor(),
    #Lambda(lambda image: image / 255),
    Lambda(lambda image: image.view(784))

])

data_train = MNIST(root='./', download=True, train=True,transform=transform)
data_test = MNIST(root='./', download=True,train=False,transform=transform)

data_train[0][0].shape

100.0%


Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw



100.0%


Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw



100.0%


Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw



100.0%

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw



torch.Size([784])

In [ ]:
from torch import nn, optim

def get_default_device():
    return torch.device('cuda') if torch.cuda_is_availavle() else torch.device('cpu')

def to_t(tensor, device=get_default_device()):
    return tensor.to(device)

class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters())
        self.to(get_default_device())

    def forward(self, X):
        return self.layers(X)
    
    def predict(self, X):
        with torch.no_grad():
            return torch.argmax(self.forward(X), axis=-1)

    def fit(self, X, Y):
        self.optimizer.zero_grad()
        y_pred = self.forward(X)
        loss = self.loss(y_pred, Y)
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [ ]:
mnist_model = MNISTModel()

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE=32
dataloader_train = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test = DataLoader(data_test, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
correct = 0
for r_x, r_y in dataloader_test:
    x=to_t(r_x)
    y=to_t(r_y)
    y_pred = mnist_model.predict(x)
    correct += (y==y_pred).sum()
acc = correct / (len(dataloader_test)*BATCH_SIZE)
print(f'ACCURACY: {acc}')

ACCURACY: 0.934005618095398


NameError: name 'torch' is not defined